In [1]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

In [40]:
# Life expectancy
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/WHOSIS_000001,WHOSIS_000015&profile=crosstable&filter=COUNTRY:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO;SEX"
life = pd.read_csv(url) 
life = life.ix[1:,0:5]
life.columns = ['Country', 'Year', 'All', 'Females', 'Males']

In [41]:
life.head(5)

,Country,Year,All,Females,Males
1,Afghanistan,2015,60.5,61.9,59.3
2,Afghanistan,2014,59.9,61.3,58.6
3,Afghanistan,2013,59.9,61.2,58.7
4,Afghanistan,2012,59.5,60.8,58.3
5,Afghanistan,2011,59.2,60.4,58.0


In [48]:
# Smoking status
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/M_Est_tob_curr,M_Est_tob_daily,M_Est_cig_curr,M_Est_cig_daily&profile=crosstable&filter=COUNTRY:*;SEX:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO;SEX"
smoking = pd.read_csv(url)
smoking = smoking.ix[1:,0:5]
smoking.columns = ['Country', 'Year', 'All', 'Females', 'Males']
smoking = smoking.replace('Not available', np.nan)

In [49]:
smoking.head(5)

,Country,Year,All,Females,Males
1,Afghanistan,2013,NaN,NaN,NaN
2,Albania,2013,29.7,8.0,51.7
3,Algeria,2013,NaN,NaN,NaN
4,Andorra,2013,33.1,28.1,38.1
5,Angola,2013,NaN,NaN,NaN


In [109]:
# Alcohol (http://apps.who.int/gho/data/node.main.A1026?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/SA_0000001400&profile=crosstable&filter=COUNTRY:*;YEAR:2015;YEAR:2014;YEAR:2013;YEAR:2012;YEAR:2011;YEAR:2010;YEAR:2009;YEAR:2008;YEAR:2007;YEAR:2006;YEAR:2005;YEAR:2004;YEAR:2003;YEAR:2002;YEAR:2001;YEAR:2000&x-sideaxis=COUNTRY;DATASOURCE;ALCOHOLTYPE&x-topaxis=GHO;YEAR"
alcohol = pd.read_csv(url)
del alcohol['Unnamed: 1']
alcohol = alcohol.ix[1:]
alcohol.columns = ['Country', 'AlcType', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008',
                  '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']

In [110]:
alcohol.head(5)

,Country,AlcType,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
1,Afghanistan,All types,NaN,NaN,0.01,0.01,0.01,0.01,0.01,0.03,0.02,0.03,0.02,0.02,0.01,0.01,0.00,0.00
2,Afghanistan,Beer,NaN,NaN,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.00,0.00,0.00,0.00
3,Afghanistan,Wine,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00
4,Afghanistan,Spirits,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.00
5,Afghanistan,Other alcoholic beverages,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [111]:
alcohol.isnull().sum()

Country      0
AlcType      0
2015       964
2014       504
2013       339
2012       342
2011        54
2010        16
2009        14
2008        14
2007        14
2006         9
2005         8
2004        14
2003        14
2002        19
2001        19
2000        18
dtype: int64

In [112]:
alcohol.dtypes

Country     object
AlcType     object
2015       float64
2014       float64
2013       float64
2012       float64
2011       float64
2010        object
2009        object
2008        object
2007        object
2006        object
2005        object
2004        object
2003        object
2002        object
2001        object
2000        object
dtype: object

In [115]:
alcohol.ix[:, 7:] = alcohol.ix[:, 7:].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 
alcohol = pd.melt(alcohol, id_vars=['Country', 'AlcType'], 
                  value_vars=['2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008',
                              '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000'])
alcohol.columns = ['Country', 'AlcType', 'Year', 'AlcConsumption']

In [116]:
alcohol.head(5)

,Country,AlcType,Year,AlcConsumption
0,Afghanistan,All types,2015,NaN
1,Afghanistan,Beer,2015,NaN
2,Afghanistan,Wine,2015,NaN
3,Afghanistan,Spirits,2015,NaN
4,Afghanistan,Other alcoholic beverages,2015,NaN


In [93]:
# Overweight 
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/NCD_BMI_25A&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR;AGEGROUP;SEX"
oweight = pd.read_csv(url)
oweight = oweight.ix[3:,0:4]
oweight.columns = ['Country', 'All', 'Females', 'Males']
oweight['Year'] = 2014

In [132]:
oweight.head(5)

,Country,All,Females,Males,Year
3,Afghanistan,16.2,19.1,13.4,2014
4,Albania,52.7,47.9,57.5,2014
5,Algeria,59.1,62.6,55.7,2014
6,Andorra,66.9,61.9,72,2014
7,Angola,30.9,37.8,23.9,2014


In [135]:
oweight.dtypes

Country     object
All        float64
Females    float64
Males      float64
Year         int64
dtype: object

In [129]:
oweight['All'] = oweight['All'].apply(lambda x: x.split(' [')[0])
oweight['Females'] = oweight['Females'].apply(lambda x: x.split(' [')[0])
oweight['Males'] = oweight['Males'].apply(lambda x: x.split(' [')[0])

In [149]:
oweight.ix[:, 1:4] = oweight.ix[:, 1:4].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [163]:
# Physical activity (http://apps.who.int/gho/data/node.main.A893?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/NCD_PAC,NCD_PAA&profile=crosstable&filter=AGEGROUP:YEARS18-PLUS;COUNTRY:*;SEX:*;&x-sideaxis=COUNTRY;YEAR;AGEGROUP&x-topaxis=GHO;SEX"
exercise = pd.read_csv(url)
del exercise['Unnamed: 2']
exercise = exercise.ix[1:, :5]
exercise.columns = ['Country', 'Year', 'All', 'Females', 'Males']

exercise['All'] = exercise['All'].apply(lambda x: x.split(' [')[0])
exercise['Females'] = exercise['Females'].apply(lambda x: x.split(' [')[0])
exercise['Males'] = exercise['Males'].apply(lambda x: x.split(' [')[0])
exercise.ix[:, 2:5] = exercise.ix[:, 2:5].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 
exercise = exercise.replace('No data', np.nan)

In [164]:
exercise.head(5)

,Country,Year,All,Females,Males
1,Afghanistan,2010,NaN,NaN,NaN
2,Albania,2010,NaN,NaN,NaN
3,Algeria,2010,34.4,41.2,27.7
4,Andorra,2010,26.1,29.7,22.4
5,Angola,2010,NaN,NaN,NaN


In [161]:
exercise.dtypes

Country     object
Year        object
All        float64
Females    float64
Males      float64
dtype: object

In [173]:
# Cholesterol (http://apps.who.int/gho/data/node.main.A884?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/CHOL_01,CHOL_02&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY;YEAR;AGEGROUP&x-topaxis=GHO;SEX"
chol = pd.read_csv(url)
del chol['Unnamed: 2']
chol = chol.ix[1:, :5]
chol.columns = ['Country', 'Year', 'All', 'Females', 'Males']

chol['All'] = chol['All'].apply(lambda x: x.split(' [')[0])
chol['Females'] = chol['Females'].apply(lambda x: x.split(' [')[0])
chol['Males'] = chol['Males'].apply(lambda x: x.split(' [')[0])
chol.ix[:, 2:5] = chol.ix[:, 2:5].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 
chol = chol.replace('No data', np.nan)

In [174]:
chol.head(5)

,Country,Year,All,Females,Males
1,Afghanistan,2008,21.9,22.8,20.9
2,Albania,2008,45.3,44.3,46.3
3,Algeria,2008,39.4,41.4,37.3
4,Andorra,2008,67.1,64.1,69.7
5,Angola,2008,30.9,31.8,29.7


In [183]:
# Blood sugar (http://apps.who.int/gho/data/node.main.A869?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/NCD_GLUC_03,NCD_GLUC_04&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY;YEAR;AGEGROUP&x-topaxis=GHO;SEX"
bsugar = pd.read_csv(url)
del bsugar['Unnamed: 2']
bsugar = bsugar.ix[1:, :5]
bsugar.columns = ['Country', 'Year', 'All', 'Females', 'Males']

bsugar['All'] = bsugar['All'].apply(lambda x: x.split(' [')[0])
bsugar['Females'] = bsugar['Females'].apply(lambda x: x.split(' [')[0])
bsugar['Males'] = bsugar['Males'].apply(lambda x: x.split(' [')[0])
bsugar.ix[:, 2:5] = bsugar.ix[:, 2:5].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 
bsugar = bsugar.replace('No data', np.nan)

In [184]:
bsugar.head(5)

,Country,Year,All,Females,Males
1,Afghanistan,2014,8.4,8.8,8.0
2,Afghanistan,2010,7.6,7.9,7.2
3,Albania,2014,8.3,8.1,8.4
4,Albania,2010,7.7,7.6,7.9
5,Algeria,2014,10.5,10.7,10.2


In [193]:
# Unsafe water, sanitation and hygiene (http://apps.who.int/gho/data/node.main.167?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/WHS5_122,WHS5_158&profile=crosstable&filter=COUNTRY:*;RESIDENCEAREATYPE:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO;RESIDENCEAREATYPE"
sanitation = pd.read_csv(url)
sanitation = sanitation[['Unnamed: 0', 'Unnamed: 1',
                        'Population using improved drinking-water sources (%).2',
                        'Population using improved sanitation facilities (%).2']]
sanitation = sanitation.ix[1:,]
sanitation.columns = ['Country', 'Year', 'ImprovedWater', 'ImprovedSanitation']

In [194]:
sanitation.head(5)

,Country,Year,ImprovedWater,ImprovedSanitation
1,Afghanistan,2015,55.3,31.9
2,Afghanistan,2000,30.3,23.4
3,Albania,2015,95.1,93.2
4,Albania,2000,96.1,83.4
5,Albania,1990,NaN,78.1


In [212]:
# Maternal deaths (http://apps.who.int/gho/data/node.main.MATMORT?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MDG_0000000026,MORT_MATERNALNUM&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO"
maternal = pd.read_csv(url)
maternal = maternal.ix[:, :3]
maternal.columns = ['Country', 'Year', 'MaternalDeaths']

maternal['MaternalDeaths'] = maternal['MaternalDeaths'].str.replace(' ', '')
maternal['MaternalDeaths'] = maternal['MaternalDeaths'].apply(lambda x: x.split('[')[0])
maternal['MaternalDeaths'] = maternal['MaternalDeaths'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [213]:
maternal.head(5)

,Country,Year,MaternalDeaths
0,Afghanistan,2015,396
1,Afghanistan,2000,1100
2,Afghanistan,1990,1340
3,Albania,2015,29
4,Albania,2000,43


In [218]:
# UV radiation exposure (http://apps.who.int/gho/data/node.main.164?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/UV_1&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR"
uvrad = pd.read_csv(url)
uvrad = uvrad.ix[1:, ]
uvrad.columns = ['Country', 'UVRadiation']

In [219]:
uvrad.head(5)

,Country,UVRadiation
1,Afghanistan,4132
2,Albania,2542
3,Algeria,3253
4,Andorra,2469
5,Angola,5287


In [225]:
# Homicide (http://apps.who.int/gho/data/node.main.VIOLENCEHOMICIDE?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/VIOLENCE_HOMICIDENUM,VIOLENCE_HOMICIDERATE&profile=crosstable&filter=COUNTRY:*;AGEGROUP:-;SEX:-&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR"
homicides = pd.read_csv(url)
del homicides['Estimates of number of homicides']
homicides = homicides.ix[1:,]
homicides.columns = ['Country', 'HomicideRate']

homicides['HomicideRate'] = homicides['HomicideRate'].apply(lambda x: x.split('[')[0])
homicides['HomicideRate'] = homicides['HomicideRate'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [226]:
homicides.head(5)

,Country,HomicideRate
1,Afghanistan,7.3
2,Albania,5.0
3,Algeria,4.4
4,Andorra,0.8
5,Angola,10.7


In [229]:
# Road traffic deaths (http://apps.who.int/gho/data/node.main.A997?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/RS_196,RS_198&profile=crosstable&filter=COUNTRY:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR"
traffdeath = pd.read_csv(url)
del traffdeath['Estimated number of road traffic deaths']
traffdeath = traffdeath.ix[1:,]
traffdeath.columns = ['Country', 'TrafficDeaths']

In [230]:
traffdeath.head(5)

,Country,TrafficDeaths
1,Afghanistan,15.5
2,Albania,15.1
3,Algeria,23.8
4,Andorra,7.6
5,Angola,26.9


In [247]:
# Malaria (http://apps.who.int/gho/data/view.main.14111?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MALARIA002&profile=crosstable&filter=COUNTRY:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR"
malaria = pd.read_csv(url)
malaria = malaria.ix[1:, :2]
malaria.columns = ['Country', 'Malaria']

malaria['Malaria'] = malaria['Malaria'].str.replace(' ', '')
malaria['Malaria'] = malaria['Malaria'].str.replace('&lt;', '')
malaria['Malaria'] = malaria['Malaria'].apply(lambda x: x.split('[')[0])
malaria['Malaria'] = malaria['Malaria'].apply(lambda x: pd.to_numeric(x, errors ='coerce')) 

In [248]:
malaria.head(5)

,Country,Malaria
1,Afghanistan,250000.0
2,Algeria,50.0
3,Angola,3400000.0
4,Argentina,0.0
5,Armenia,NaN


In [249]:
# Number of people with HIV (http://apps.who.int/gho/data/node.main.620?lang=en)
url = "http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/HIV_0000000001&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR"
hiv = pd.read_csv(url)